### Frontend for AD9255

This notebook shows how to calculate the parameters of the transformer-coupled frontend using the process described in [AN-935](https://www.analog.com/en/resources/app-notes/an-935.html).

Import the required libraries.

In [ ]:
import numpy as np
import pylab as pl

Read data.

In [ ]:
adc = np.loadtxt("AD9255.txt")

f = adc[:, 0]

Rt = adc[:, 1]
Ct = adc[:, 2]

Rh = adc[:, 3]
Ch = adc[:, 4]

tr = np.loadtxt("TC4-1T+.txt")

ftr = tr[:, 0]
RLtr = tr[:, 2]

def RLcorr(f):
    RL = -np.interp(f, ftr, RLtr)
    return (1 - 10 ** (RL / 20)) / (1 + 10 ** (RL / 20))

Plot data.

In [ ]:
pl.plot(f, Rt, label="track mode")
pl.plot(f, Rh, label="hold mode")

pl.ylabel("kOhm")
pl.xlabel("MHz")

pl.ylim(0, 30)
pl.xlim(0, 100)

pl.grid()
pl.legend()
pl.show()

In [ ]:
pl.plot(f, Ct, label="track mode")
pl.plot(f, Ch, label="hold mode")

pl.ylabel("pF")
pl.xlabel("MHz")

pl.ylim(0, 16)
pl.xlim(0, 100)

pl.grid()
pl.legend()
pl.show()

Frontend with inductance.

In [ ]:
def Z(f, RA, CA, L, RS, RP):
    ZCA = 1 / (1j * 2 * np.pi * f * CA)
    ZA = 1 / (1 / RA + 1 / ZCA)
    ZL = 1j * 2 * np.pi * f * L
    ZALRS = 1 / (1 / ZA + 1 / ZL) + RS * 2
    ZALRSRP = 1 / (1 / ZALRS + 1 / RP)
    return ZALRSRP


corr = RLcorr(f)

result = Z(f * 1e6, Rt * 1e3, Ct * 1e-12, 33e-6, 33, 220) * corr / 4

pl.plot(f, np.real(result), label="Re(Z)")
pl.plot(f, np.imag(result), label="Im(Z)")

pl.ylabel("Ohm")
pl.xlabel("MHz")

pl.ylim(-20, 60)
pl.xlim(0, 100)

pl.grid()
pl.legend()
pl.show()

Frontend without inductance.

In [ ]:
def Z1(f, RA, CA, RS, RP):
    ZCA = 1 / (1j * 2 * np.pi * f * CA)
    ZA = 1 / (1 / RA + 1 / ZCA)
    ZARS = ZA + RS * 2
    ZARSRP = 1 / (1 / ZARS + 1 / RP)
    return ZARSRP


corr = RLcorr(f)

result = Z1(f * 1e6, Rt * 1e3, Ct * 1e-12, 33, 220) * corr / 4

pl.plot(f, np.real(result), label="Re(Z)")
pl.plot(f, np.imag(result), label="Im(Z)")
pl.plot(f, np.absolute(result), label="|Z|")

pl.ylabel("Ohm")
pl.xlabel("MHz")

pl.ylim(-20, 70)
pl.xlim(0, 100)

pl.grid()
pl.legend()
pl.show()

In [ ]:
def Z2(f, RA, CA, RS, RP):
    ZCA = 1 / (1j * 2 * np.pi * f * CA)
    ZA = 1 / (1 / RA + 1 / ZCA)
    ZARP = 1 / (1 / ZA + 1 / RP)
    ZARPRS = ZARP + RS * 2
    return ZARPRS


corr = RLcorr(f)

result = Z2(f * 1e6, Rt * 1e3, Ct * 1e-12, 10, 200) * corr / 4

pl.plot(f, np.real(result), label="Re(Z)")
pl.plot(f, np.imag(result), label="Im(Z)")
pl.plot(f, np.absolute(result), label="|Z|")
pl.ylabel("Ohm")
pl.xlabel("MHz")

pl.ylim(-20, 70)
pl.xlim(0, 100)

pl.grid()
pl.legend()
pl.show()

In [ ]:
def F1(f, RA, CA, RS, RP):
    ZCA = 1 / (1j * 2 * np.pi * f * CA)
    ZA = 1 / (1 / RA + 1 / ZCA)
    ZARS = ZA + RS * 2
    ZARSRP = 1 / (1 / ZARS + 1 / RP)
    return np.absolute(ZARSRP / (ZARSRP + 220) * ZA / ZARS)


def F2(f, RA, CA, RS, RP):
    ZCA = 1 / (1j * 2 * np.pi * f * CA)
    ZA = 1 / (1 / RA + 1 / ZCA)
    ZARP = 1 / (1 / ZA + 1 / RP)
    ZARPRS = ZARP + RS * 2
    return np.absolute(ZARPRS / (ZARPRS + 220) * ZARP / ZARPRS)


r1 = F1(f * 1e6, Rt * 1e3, Ct * 1e-12, 33, 220)
r2 = F2(f * 1e6, Rt * 1e3, Ct * 1e-12, 10, 200)

pl.plot(f, 20 * np.log10(r1), label="F1")
pl.plot(f, 20 * np.log10(r2), label="F2")

pl.xlabel("MHz")

pl.ylim(-9, -4)
pl.xlim(0, 100)

pl.grid()
pl.legend()
pl.show()